In [1]:
from gensim.models import FastText

In [2]:
from gensim.models.fasttext import ft_ngram_hashes  # This function is used to calculate hashes from ngrams to determine position in ngram matrix
from gensim.models.fasttext_inner import compute_ngrams_bytes, compute_ngrams

In [3]:
import os
import tqdm
import numpy as np
import gensim

from collections import defaultdict

### Loading fasttext model

In [4]:
ft = gensim.models.KeyedVectors.load("models/fasttext/wiki.ru_gensim.model")

In [5]:
ft.vectors.shape

(1888423, 300)

In [6]:
ft.vectors_ngrams.shape

(2000000, 300)

In [7]:
# Данные значения корректируются по коду ниже
new_vocab_size = 80_000
new_ngrams_size = 100_000  # Should be GCD of initial 

### New vocab
Here we select the most frequent words in existing vocabulary

In [8]:
file = open("models/train_input.txt", "r")

# считываем все строки
all_text = file.readlines()
file.close()

In [9]:
texts = [[text for text in doc.split()] for doc in all_text]

In [10]:
#Обучение используется лишь для получения всех токинов по системе fasttext
model = FastText(texts, vector_size=100, window=5, min_count=1)

In [11]:
len(model.wv.index_to_key)

4753

In [12]:
#Второй способ
# from sklearn.feature_extraction.text import TfidfVectorizer

# file = open("models/train_input.txt", "r")

# # считываем все строки
# all_text = file.readlines()
# file.close()

# tfidf = TfidfVectorizer(token_pattern = r"(?u)\b\w+\b")
# tfidf.fit(all_text)

# list(tfidf.__dict__)

In [13]:
#Заполним сначала словами встречающимися в тексте
ntop = 10
all_list_words = []
not_in_list = []
osn_word = 0
add_word = 0
for word in tqdm.tqdm(model.wv.index_to_key):
    if word in ft.key_to_index:
        all_list_words.append(word)
        osn_word += 1
    else:
        not_in_list.append(word)
        
    list_words = list(ft.most_similar(word, ntop))    
    for word, score in list_words:
        if word in ft.key_to_index:
            all_list_words.append(word)
            add_word += 1
        
osn_word, add_word

100%|██████████| 4753/4753 [11:56<00:00,  6.64it/s]


(2652, 47530)

In [14]:
#Уникальный слов всего, до добавления нграмм
all_list_words = list(set(all_list_words))
len(all_list_words)

32213

In [15]:
list_all_ngramm = []

In [16]:
#Добавляем нграммы слов, которых не было в словаре
for word in not_in_list:
    encoded_ngrams = compute_ngrams(word, 1, ft.max_n)
    for ngram in encoded_ngrams:
        if ngram in ft.key_to_index:
            list_all_ngramm.append(ngram)

In [17]:
#Добавляем нграммы остальных слов
for word in all_list_words:
    encoded_ngrams = compute_ngrams(word, 1, ft.max_n)
    for ngram in encoded_ngrams:
        if ngram in ft.key_to_index:
            list_all_ngramm.append(ngram)

In [18]:
#Минимальное количество необходимых нграмм
#Данное знаечение или больше необходимо записать в переменную new_ngrams_size
list_all_ngramm = list(set(list_all_ngramm))
len(list_all_ngramm)

50532

In [19]:
#Минимальное количество слов в словаре, лучше чуть больше сделать
#Данное знаечение или больше необходимо записать в переменную new_vocab_size
all_list_words.extend(list_all_ngramm)
all_list_words = list(set(all_list_words))
len(all_list_words)

72778

In [20]:
#Итоговый список всех слов
len(all_list_words)

72778

In [21]:
#Получим индексы слов из словаря
list_ft_index_word = []
for word in all_list_words:
    ft_index = ft.key_to_index[word]
    list_ft_index_word.append(ft_index)
list_check_word = all_list_words
del(all_list_words)
len(list_ft_index_word)

72778

In [22]:
#Сортируем список индексов слов в словаре ft
list_ft_index_word = sorted(list_ft_index_word)

In [23]:
#Будет добавлено дополнительно частовстречаемых слов из основной модели
add_vocab_size = max(0,new_vocab_size - len(list_ft_index_word))
add_vocab_size

7222

In [24]:
#Добавляем частовстречаемые слова в наш словарь
for idx in range(new_vocab_size):
    word = ft.index_to_key[idx]
    if idx not in list_ft_index_word:
        list_ft_index_word.append(idx)
    if len(list_ft_index_word) >= new_vocab_size:
        break
len(list_ft_index_word)

80000

In [25]:
#Сортируем список по индексам в словаре ft еще раз
list_ft_index_word = sorted(list_ft_index_word)

In [26]:
len(list_ft_index_word)

80000

In [27]:
#Создаем итоговый словрь
top_vocab = {}
list_top_vocab_vectors = []
for idx in list_ft_index_word:
    top_vocab[ft.index_to_key[idx]] = idx
    list_top_vocab_vectors.append(ft.vectors_vocab[idx])
#top_vocab
top_vocab_vectors = np.array(list_top_vocab_vectors)
top_vocab_vectors.shape

(80000, 300)

In [28]:
#Проверим что количество полученных слов равно необходимому количеству слов
assert new_vocab_size == len(list_ft_index_word)

### Ngrams remapping

Ngram vectors are located by calculating hash of the ngram chars.
We need to calculate new hashes for smaller matrix and map old vectors to a new ones.
Since the size of the new matrix is smaller, there will be collisions.
We are going to resolv them by calculating weighted sum of vectors of collided ngrams.

In [29]:
new_to_old_buckets = defaultdict(set)
old_hash_count = defaultdict(int)
for word, vocab_word in tqdm.tqdm(top_vocab.items()):
    old_hashes = ft_ngram_hashes(word, ft.min_n, ft.max_n, ft.bucket)
    new_hashes = ft_ngram_hashes(word, ft.min_n, ft.max_n, new_ngrams_size)
    
    for old_hash in old_hashes:
        old_hash_count[old_hash] += 1  # calculate frequency of ngrams for proper weighting
        
    for old_hash, new_hash in zip(old_hashes, new_hashes):
        new_to_old_buckets[new_hash].add(old_hash)

100%|██████████| 80000/80000 [00:01<00:00, 42109.74it/s]


In [30]:
def fasttext_like_init(n, dim=300, random_state=42):
    rand_obj = np.random
    rand_obj.seed(random_state)
    lo, hi = -1.0 / dim, 1.0 / dim
    new_ngrams = rand_obj.uniform(lo, hi, (n, dim)).astype(np.float32)
    return new_ngrams

In [31]:
# Create new FastText model instance
new_ft = gensim.models.keyedvectors.FastTextKeyedVectors(
    vector_size=ft.vector_size,
    min_n=ft.min_n,
    max_n=ft.max_n,
    bucket=new_ngrams_size
)

In [32]:
#Для новой модели, сбросим индексы, при этом оставим прежний порядок слов в словаре
new_top_vocab = top_vocab.copy()
for idx, key in enumerate(new_top_vocab.keys()):
    new_top_vocab[key] = idx

In [33]:
# Set shrinked vocab and vocab vector
new_ft.vectors_vocab = top_vocab_vectors
new_ft.vectors = new_ft.vectors_vocab
new_ft.key_to_index = new_top_vocab
new_ft.index_to_key = list(top_vocab)

In [34]:
#Заполним полученное количество нграмм
len(new_to_old_buckets)

96669

In [35]:
new_ngrams_size

100000

In [36]:
# Set ngram vectors
new_ngrams = fasttext_like_init(n=new_ngrams_size, dim=ft.vectors_ngrams.shape[1], random_state=42)
for new_hash, old_buckets in new_to_old_buckets.items():
    total_sum = sum(old_hash_count[old_hash] for old_hash in old_buckets)
    
    new_vector = np.zeros(ft.vector_size, dtype=np.float32)
    for old_hash in old_buckets:
        weight = old_hash_count[old_hash] / total_sum
        new_vector += ft.vectors_ngrams[old_hash] * weight
    

    new_ngrams[new_hash] = new_vector
new_ft.vectors_ngrams = new_ngrams

In [37]:
#Перезагрузим модель, чтобы произошло обновление таблицы вектров модели
new_name_model = 'models/fasttext/shrinked_fasttext.model'
new_ft.save(new_name_model)
new_ft = gensim.models.KeyedVectors.load(new_name_model)
new_ft 

### Calculating losses

We may gain a view of accuracy losses by measuring similarity between original vocab vectors and new vectors recreated from shrink n-grams.

In [38]:
#Проверим аналогичность новой модели путем запроса похожих слов, должна выдавать похожие результаты

In [39]:
ft.most_similar('изолента', topn=10)

[('изоленты', 0.8522964715957642),
 ('изоленту', 0.8454886078834534),
 ('изолентой', 0.8099086284637451),
 ('изоле', 0.7382572293281555),
 ('полента', 0.6976317167282104),
 ('солента', 0.6515787243843079),
 ('изолы', 0.6471081376075745),
 ('полента\xa0—', 0.6222870945930481),
 ('изол', 0.6166868805885315),
 ('перфолента', 0.6091929078102112)]

In [40]:
new_ft.most_similar('изолента', topn=10)

[('изоленты', 0.8465209603309631),
 ('изоленту', 0.8347629308700562),
 ('изолентой', 0.8115509152412415),
 ('изоле', 0.7830286622047424),
 ('полента', 0.7714607119560242),
 ('изола', 0.7516038417816162),
 ('изол', 0.7508651614189148),
 ('тизола', 0.6680089831352234),
 ('тизол', 0.646018385887146),
 ('черизоле', 0.642540693283081)]

In [41]:
new_test_vocab = list_check_word

In [42]:
#ft.rank_by_centrality('изолента')

In [43]:
#Качество исходной модели составления из n-грамм
sims = []
old_word = ''
for test_word in new_test_vocab:
    str_word = test_word + '' + old_word
    sim = ft.cosine_similarities(ft.get_vector(test_word) + ft.get_vector(old_word), [ft.get_vector(str_word)])
    old_word = word
    if not np.isnan(sim):
        sims.append(sim)
print(f"size: {new_ngrams_size}", "Similarity:", np.sum(sims) / len(new_test_vocab))

size: 100000 Similarity: 0.742301019452994


In [44]:
#Качество новой модели составления из n-грамм
sims = []
old_word = ''
for test_word in new_test_vocab:
    str_word = test_word + '' + old_word
    sim = ft.cosine_similarities(new_ft.get_vector(test_word) + new_ft.get_vector(old_word), [new_ft.get_vector(str_word)])
    old_word = word
    if not np.isnan(sim):
        sims.append(sim)
print(f"size: {new_ngrams_size}", "Similarity:", np.sum(sims) / len(new_test_vocab))

size: 100000 Similarity: 0.7300436022046498


In [45]:
#Сравнение сходства двух моделей по основной методике
sims = []
for test_word in new_test_vocab:
    sim = ft.cosine_similarities(ft.get_vector(test_word), [new_ft.get_vector(test_word)])
    if not np.isnan(sim):
        sims.append(sim)
print(f"size: {new_ngrams_size}", "Similarity:", np.sum(sims) / len(new_test_vocab))

size: 100000 Similarity: 0.824769762153398


In [46]:
#Сравнение сходства двух моделей по модифицированной методике
sims = []
old_word = ''
for test_word in new_test_vocab:
    str_word = test_word + '' + old_word
    sim = ft.cosine_similarities(ft.get_vector(str_word), [new_ft.get_vector(str_word)])
    old_word = word
    if not np.isnan(sim):
        sims.append(sim)
print(f"size: {new_ngrams_size}", "Similarity:", np.sum(sims) / len(new_test_vocab))

size: 100000 Similarity: 0.8457632393202617


#### 